In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atherosclerosis"
cohort = "GSE154851"

# Input paths
in_trait_dir = "../../input/GEO/Atherosclerosis"
in_cohort_dir = "../../input/GEO/Atherosclerosis/GSE154851"

# Output paths
out_data_file = "../../output/preprocess/Atherosclerosis/GSE154851.csv"
out_gene_data_file = "../../output/preprocess/Atherosclerosis/gene_data/GSE154851.csv"
out_clinical_data_file = "../../output/preprocess/Atherosclerosis/clinical_data/GSE154851.csv"
json_path = "../../output/preprocess/Atherosclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Investigation Of Genes Associated With Atherosclerosis In Patients With Systemic Lupus Erythematosus"
!Series_summary	"Systemic lupus erythematosus (SLE) is a chronic, autoimmune disease affecting multiple heterogeneous organs and systems. SLE is associated with increased risk of atherosclerosis and increased cardiovascular complications. In this study, we specifically aimed to identify patients with SLE who are genetically at risk for developing atherosclerosis. Sureprint G3 Human Gene Expression 8x60K Microarray kit (Agilent technologies, Santa Clara, CA, USA) was used in our study. Genes showing differences in expression between the groups were identified by using GeneSpring GX 10.0 program. A total of 155 genes showing expression level difference were detected between SLE patients and healthy controls. In molecular network analysis."
!Series_overall_design	"38 patients with systemic lupus erythematosus (36 females, 2 males) and 32 healthy cont

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
"""
Analysis:
- This dataset appears to study atherosclerosis in SLE patients vs healthy controls
- It uses gene expression microarray data, which is suitable for our analysis
- Sample characteristics include:
  - Gender (Key 1): mostly female with some male participants
  - Age (Key 2): ranges from 18 to 71 years
  - Disease status: Not explicitly in sample characteristics, but from the background
    information we can infer SLE status, which is relevant to the trait (atherosclerosis)
"""

# 1. Gene Expression Data Availability
# This dataset uses "Sureprint G3 Human Gene Expression 8x60K Microarray kit"
is_gene_available = True

# 2.1 Data Availability
# From the background, we know SLE patients may have atherosclerosis risk
# While atherosclerosis itself is not explicitly coded, we can use the SLE status as a proxy
# (atherosclerosis is a complication of SLE according to the background)
trait_row = None  # No explicit atherosclerosis data
age_row = 2  # Age information is available at key 2
gender_row = 1  # Gender information is available at key 1

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    # No direct atherosclerosis data
    return None

def convert_age(value):
    # Extract numerical age from string like "age: 18y"
    try:
        if isinstance(value, str) and 'age:' in value:
            # Extract the number from strings like "age: 18y"
            age_str = value.split(':')[1].strip()
            return int(age_str.replace('y', ''))
        return None
    except:
        return None

def convert_gender(value):
    # Convert gender values to binary (0=female, 1=male)
    try:
        if isinstance(value, str) and 'gender:' in value:
            gender = value.split(':')[1].strip().lower()
            if gender == 'female':
                return 0
            elif gender == 'male':
                return 1
        return None
    except:
        return None

# 3. Save Metadata
# Initial filtering to check if dataset is usable
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep
# However, we can still extract age and gender information
if trait_row is not None:
    # This block won't execute but is kept for completeness
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atherosclerosis/GSE154851/GSE154851_series_matrix.txt.gz


Gene data shape: (62976, 70)
First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers shown are numeric ('1', '2', '3', etc.) which are not human gene symbols.
# These are likely probe IDs or internal identifiers that need to be mapped to gene symbols.
# For proper biological interpretation, we need to map these to standard gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's explore the SOFT file more thoroughly to find gene symbols
print("\nExploring SOFT file more thoroughly for gene information:")
gene_info_patterns = []
entrez_to_symbol = {}

with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if i < 1000:  # Check header section for platform info
            if '!Series_platform_id' in line or '!Platform_title' in line:
                print(line.strip())
                
        # Look for gene-related columns and patterns in the file
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line or 'Symbol' in line:
            gene_info_patterns.append(line.strip())
            
        # Extract a mapping using ENTREZ_GENE_ID if available
        if len(gene_info_patterns) < 2 and 'ENTREZ_GENE_ID' in line and '\t' in line:
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                try:
                    # Attempt to add to mapping - assuming ENTREZ_GENE_ID could help with lookup
                    entrez_id = parts[1]
                    probe_id = parts[0]
                    if entrez_id.isdigit() and entrez_id != probe_id:
                        entrez_to_symbol[probe_id] = entrez_id
                except:
                    pass
        
        if i > 10000 and len(gene_info_patterns) > 0:  # Limit search but ensure we found something
            break

# Show some of the patterns found
if gene_info_patterns:
    print("\nFound gene-related patterns:")
    for pattern in gene_info_patterns[:5]:
        print(pattern)
else:
    print("\nNo explicit gene info patterns found")

# Let's try to match the ENTREZ_GENE_ID to the probe IDs
print("\nAnalyzing ENTREZ_GENE_ID column:")
if 'ENTREZ_GENE_ID' in gene_annotation.columns:
    # Check if ENTREZ_GENE_ID contains actual Entrez IDs (different from probe IDs)
    gene_annotation['ENTREZ_GENE_ID'] = gene_annotation['ENTREZ_GENE_ID'].astype(str)
    different_ids = (gene_annotation['ENTREZ_GENE_ID'] != gene_annotation['ID']).sum()
    print(f"Number of entries where ENTREZ_GENE_ID differs from ID: {different_ids}")
    
    if different_ids > 0:
        print("Some ENTREZ_GENE_ID values differ from probe IDs - this could be useful for mapping")
        # Show examples of differing values
        diff_examples = gene_annotation[gene_annotation['ENTREZ_GENE_ID'] != gene_annotation['ID']].head(5)
        print(diff_examples)
    else:
        print("ENTREZ_GENE_ID appears to be identical to probe ID - not useful for mapping")

# Search for additional annotation information in the dataset
print("\nLooking for alternative annotation approaches:")
print("- Checking for platform ID or accession number in SOFT file")

platform_id = None
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Platform_geo_accession' in line:
            platform_id = line.split('=')[1].strip().strip('"')
            print(f"Found platform GEO accession: {platform_id}")
            break
        if i > 200:
            break

# If we don't find proper gene symbol mappings, prepare to use the ENTREZ_GENE_ID as is
if 'ENTREZ_GENE_ID' in gene_annotation.columns:
    print("\nPreparing provisional gene mapping using ENTREZ_GENE_ID:")
    mapping_data = gene_annotation[['ID', 'ENTREZ_GENE_ID']].copy()
    mapping_data.rename(columns={'ENTREZ_GENE_ID': 'Gene'}, inplace=True)
    print(f"Provisional mapping data shape: {mapping_data.shape}")
    print(preview_df(mapping_data, n=5))
else:
    print("\nWarning: No suitable mapping column found for gene symbols")



Gene annotation preview:
Columns in gene annotation: ['ID', 'COL', 'ROW', 'NAME', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'LOCUSLINK_ID', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# From the previous analysis, we can see:
# - Gene identifiers in gene expression data: numeric IDs like '1', '2', '3' which correspond to the 'ID' column in gene_annotation
# - Gene symbols are in the 'GENE_SYMBOL' column of gene_annotation

# 2. Create a gene mapping dataframe
# We need to map from 'ID' to 'GENE_SYMBOL'
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Mapping data shape: {mapping_data.shape}")
print("First 5 rows of mapping data:")
print(mapping_data.head())

# 3. Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5])

# Let's normalize gene symbols to standard format
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First 5 normalized gene symbols:")
print(gene_data.index[:5])

# Save the gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Mapping data shape: (54295, 2)
First 5 rows of mapping data:
  ID          Gene
3  4         HEBP1
4  5         KCNE4
5  6        BPIFA3
6  7  LOC100129869
7  8          IRG1
Gene expression data shape after mapping: (20353, 70)
First 5 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M'], dtype='object', name='Gene')


Gene expression data shape after normalization: (19847, 70)
First 5 normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1'], dtype='object', name='Gene')


Gene expression data saved to: ../../output/preprocess/Atherosclerosis/gene_data/GSE154851.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI database
print("Normalizing gene symbols...")
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the normalized gene data
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 2. Extract and prepare clinical data from the matrix file
print("\nPreparing clinical data...")
_, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Since Step 2 identified no atherosclerosis trait data is available,
# but we still need to correctly extract age and gender data for completeness
# Define conversion functions for age and gender
def convert_age(value):
    try:
        if isinstance(value, str) and 'age:' in value:
            # Extract the number from strings like "age: 18y"
            age_str = value.split(':')[1].strip()
            return int(age_str.replace('y', ''))
        return None
    except:
        return None

def convert_gender(value):
    try:
        if isinstance(value, str) and 'gender:' in value:
            gender = value.split(':')[1].strip().lower()
            if gender == 'female':
                return 0
            elif gender == 'male':
                return 1
        return None
    except:
        return None

# Extract demographic data we do have
age_row = 2    # Row for age information
gender_row = 1  # Row for gender information

# Create a processed clinical dataframe with at least demographic information
clinical_df = pd.DataFrame()
if age_row is not None:
    age_data = get_feature_data(clinical_data, age_row, 'Age', convert_age)
    clinical_df = pd.concat([clinical_df, age_data], axis=0) if not clinical_df.empty else age_data
if gender_row is not None:
    gender_data = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender)
    clinical_df = pd.concat([clinical_df, gender_data], axis=0) if not clinical_df.empty else gender_data

# Add an empty Atherosclerosis column to maintain the expected structure
atherosclerosis_row = pd.DataFrame([pd.Series(float('nan'), index=clinical_df.columns)], index=[trait])
clinical_df = pd.concat([atherosclerosis_row, clinical_df], axis=0)

print("Processed clinical data preview:")
print(preview_df(clinical_df))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to: {out_clinical_data_file}")

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 samples, 5 features):")
print(linked_data.iloc[:5, :5] if linked_data.shape[0] > 0 and linked_data.shape[1] > 5 else linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
# Since we know we don't have trait data, we'll set is_biased to False as it's neither biased nor unbiased
# The validation function still needs to receive a parameter though
is_biased = False

# 6. Conduct final quality validation
note = "This GSE154851 dataset contains gene expression data from SLE patients with increased risk of atherosclerosis. While gene expression data is available, explicit atherosclerosis trait data is not provided, making the dataset unsuitable for our specific atherosclerosis association study."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # No trait data available
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

print("\nDataset validation complete.")
print(f"Dataset usable for association studies: {is_usable}")
print(f"Note: {note}")

# 7. Don't save the linked data since it's not usable (no trait information)
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: (19847, 70)
First 10 normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT',
       'AA06', 'AAA1'],
      dtype='object', name='Gene')


Normalized gene data saved to: ../../output/preprocess/Atherosclerosis/gene_data/GSE154851.csv

Preparing clinical data...


Processed clinical data preview:
{'GSM4681537': [nan, 18.0, 0.0], 'GSM4681538': [nan, 37.0, 0.0], 'GSM4681539': [nan, 59.0, 0.0], 'GSM4681540': [nan, 36.0, 0.0], 'GSM4681541': [nan, 56.0, 0.0], 'GSM4681542': [nan, 22.0, 0.0], 'GSM4681543': [nan, 53.0, 0.0], 'GSM4681544': [nan, 41.0, 1.0], 'GSM4681545': [nan, 33.0, 0.0], 'GSM4681546': [nan, 52.0, 0.0], 'GSM4681547': [nan, 42.0, 0.0], 'GSM4681548': [nan, 28.0, 0.0], 'GSM4681549': [nan, 45.0, 0.0], 'GSM4681550': [nan, 41.0, 0.0], 'GSM4681551': [nan, 25.0, 0.0], 'GSM4681552': [nan, 34.0, 0.0], 'GSM4681553': [nan, 40.0, 0.0], 'GSM4681554': [nan, 44.0, 0.0], 'GSM4681555': [nan, 42.0, 0.0], 'GSM4681556': [nan, 39.0, 0.0], 'GSM4681557': [nan, 51.0, 0.0], 'GSM4681558': [nan, 41.0, 0.0], 'GSM4681559': [nan, 52.0, 0.0], 'GSM4681560': [nan, 34.0, 0.0], 'GSM4681561': [nan, 21.0, 0.0], 'GSM4681562': [nan, 23.0, 0.0], 'GSM4681563': [nan, 32.0, 0.0], 'GSM4681564': [nan, 39.0, 0.0], 'GSM4681565': [nan, 71.0, 0.0], 'GSM4681566': [nan, 23.0, 0.0], 'GSM46